In [ ]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2

In [ ]:
weights_path = "../input/efficientnet-v2-raw-model/EfficientNet_V2_raw.h5"
hub_url = '../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-l-21k-ft1k/feature_vector'

In [ ]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape = [HEIGHT, WIDTH, CHANNELS]),
            hub.KerasLayer(hub_url, trainable = True),
            Dropout(top_dropout_rate, name = "top_dropout"),
            Dense(CLASSES, activation = 'sigmoid')
        ])
    model.build((HEIGHT, WIDTH, CHANNELS))
    model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model

In [ ]:
model = get_model()
model.load_weights(weights_path)

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission

In [ ]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(test_img+'/'+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def process(img):
    return cv2.resize(img / 255.0 , (480, 480)).reshape(-1, 480, 480, 3)
def predict(img):
    img = load_image(img)
    tta_steps = 5
    predictions = []
    for i in range(tta_steps):
        pred = model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]
        predictions.append(pred)
    result = np.median(predictions, axis=0)
    
    return result

In [ ]:
test_images = submission['image'][2]
test_images = load_image(test_images)
preds = model.layers[2](model.layers[1](model.layers[0](process(test_images)))).numpy()[0]
print(preds[4])

In [ ]:
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
label = []
for i in range(len(submission['image'])):
    test_images = submission['image'][i]
    preds = predict(test_images)
    answer = []
    for j in range(len(preds)):
        if preds[j] > 0.3:
            answer.append(n_label[j])
    if ("healthy" in answer) and (len(answer) > 1):
        answer.remove('healthy')
    if len(answer) == 0:
        answer.append('healthy')
    answer = ' '.join(answer)
    label.append(answer)
submission['labels'] = label
submission

In [ ]:
submission.to_csv('submission.csv', index=False)